# Wine quality prediction

This notebook builds a model to predict the quality of wine using the model [sklearn.linear_model.ElasticNet](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html) and [dataset](http://archive.ics.uci.edu/ml/datasets/Wine+Quality).


## Train the best model
Use the Ezmeral Training cluster configured with higher resource to train the model with big data on the selected hyperparameter (alpha=0.5, l1_ratio=0.5).

In [ ]:
# Task: Run a magic to list the training clusters that are attached to the notebook server
%

In [ ]:
%%capture history_url

%% # Task: Run a magic to select the training cluster to submit the following code to the training cluster

import os
import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
    
import mlflow.sklearn
from mlflow import log_metric, log_param, log_artifact

def train(in_alpha, in_l1_ratio, data_path=None):


    def eval_metrics(actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2


    warnings.filterwarnings("ignore")
    np.random.seed(40)
     
    # Read from shared data volume (make sure to put the data in the Project Repository on ECP)
    data_path = "/bd-fs-mnt/project_repo/data/wine-quality.csv"

    data = pd.read_csv(data_path)
    

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]

    # Set default values if no alpha is provided
    if float(in_alpha) is None:
        alpha = 0.5
    else:
        alpha = float(in_alpha)

    # Set default values if no l1_ratio is provided
    if float(in_l1_ratio) is None:
        l1_ratio = 0.5
    else:
        l1_ratio = float(in_l1_ratio)
    
    # Execute ElasticNet
    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(train_x, train_y)

    # Evaluate Metrics
    predicted_qualities = lr.predict(test_x)
    (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

    # Print out metrics
    print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
    print("  RMSE: %s" % rmse)
    print("  MAE: %s" % mae)
    print("  R2: %s" % r2)

    # Log parameter, metrics, and model to MLflow
    mlflow.set_experiment('winequality_experiment')
    mlflow.log_param("alpha", alpha)
    mlflow.log_param("l1_ratio", l1_ratio)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)

    mlflow.sklearn.log_model(lr, "model")
    
    mlflow.end_run()
    
train(0.5, 0.5)


In [ ]:
import time

historyurl = history_url.stdout.split(' ')[2]

%logs --url $historyurl
time.sleep(5) # wait here for the training to finish to see the final log
%logs --url $historyurl
# rerun this cell if you don't see "Job Status" updates